In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['text.usetex'] = True
import ipywidgets as widgets
import matplotlib.animation as ani
from matplotlib import rc
from IPython.display import HTML, display
rcParams['animation.html'] = 'html5'

def read_history(name):

    dct = {}
    f = open(name)
    for i, line in enumerate(f):
        if i == 5:
            keys = line.split()
            break
    f.close()
    data = np.genfromtxt(name,skip_header=5)
    data= data[~np.isnan(data).any(axis=1)]
    
    for j, key in enumerate(keys):
        dct[key] = data[:,j]

    return dct

In [2]:
evol = widgets.Checkbox(
    value=True,
    description='Turn on evolution:',
    disabled=False
)
display(evol)

Checkbox(value=True, description='Turn on evolution:')

In [3]:
dct = read_history("history.data")
age = dct["star_age"]/1e6
M = dct["star_mass"]
R = dct["photosphere_r"]
L = dct["luminosity"]
Teff = dct["effective_T"]
numax = 3090*M/R**2/np.sqrt(Teff)

In [8]:
if not evol.value:
    setage = widgets.FloatSlider(
        value=max(age)/2,
        min=min(age),
        max=max(age),
        step=0.1,
        description='Age [Myr]:',
        disabled=False,
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
    )
    display(setage)

In [16]:
fig, ax = plt.subplots()
ax.semilogy(Teff,L,lw=2)
plt.gca().invert_xaxis()
plt.xlabel(r'$T_\mathrm{eff} \,\, \mathrm{[K]}$',fontsize=16)
plt.ylabel(r'$L \,\, [\mathrm{L_\odot}]$',fontsize=16)
plt.xticks(size=16)
plt.yticks(size=16)
plt.gcf().subplots_adjust(bottom=0.18)
plt.gcf().subplots_adjust(left=0.18)

if not evol.value:
    iage = abs(age-setage.value).argmin()
    ax.semilogy(dct["effective_T"][iage],dct["luminosity"][iage],"ro")
else:
    ages = np.linspace(min(age),max(age),200)
    pt, = ax.plot([], [], "ro")
    def init():
        pt.set_data([], [])
        return (pt,)
    def animate(i):
        iage = abs(age-ages[i]).argmin()
        x = Teff[iage]
        y = L[iage]
        pt.set_data(x, y)
        return (pt,)    
    anim = ani.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=50, blit=True)
    
    video = anim.to_html5_video()
    html = HTML(video)
    display(html)
    plt.close()
    
    